# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Additional web scraping of online reviews

During our EDA, we noticed two main trends in the distribution of our dataset:
1. Less than 10% of our reviews were published from the years 2022 to 2024, making it hard for us to capture recent trends in sentiment.
2. Most of the reviews were highly positive, which could mean that SIA had mostly positive reviews, nevertheless we wanted to get more information on negative reviews to improve the robustness of our model.

### TripAdvisor

We scraped more data for airline reviews from TripAdvisor, specifically for the years 2022 to 2024. 
(https://www.tripadvisor.com.sg/Airline_Review-d8729151-Reviews-Singapore-Airlines)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 5)


### Skytrax

We also scraped from Skytrax, which is another data source for online reviews. 
(https://www.airlinequality.com/airline-reviews/singapore-airlines/?sortby=post_date%3ADesc&pagesize=100)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 10)

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [1]:
!pip3 install -r requirements.txt

In [20]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime 

# Statistical functions
from scipy.stats import zscore

# Text Preprocessing and NLP
import nltk
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer


# For generating n-grams
from nltk.util import ngrams
from collections import Counter

## Data Preparation (Loading CSV)

Load the three CSV files into a pandas DataFrame `data`.

In [21]:
data = pd.read_csv('final_df.csv')

In [22]:
data.head()

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...


In [23]:
data['sentiment'].value_counts()

sentiment
Positive    7913
Negative    2441
Neutral     1164
Name: count, dtype: int64

In [24]:
data['year'].value_counts()

year
2019    5129
2018    2596
2022    1184
2023    1111
2020     888
2024     514
2021      96
Name: count, dtype: int64

# ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)

### Replaced Token Detection (RTD)
- Unlike BERT, which masks words in the input and then tries to predict the, ELECTRA randomly replaces certain tokens with plausible alternatives generated by a small generator model and trains a larger discriminator model to detect whether each token is "real" (original) or "replaced" (fake).

- This setup means that every token in the input sequence is used during training, not just the masked ones, which leads to more efficient training.

### Two-Part Architecture
- **Generator:** A smaller model (often a smaller BER) that replaces tokens with plausible alternatives. It essentially "corrupts" the input sentence by substituting some tokens with similar words.

- **Discriminator:** The main ELECTRA model, which learns to classify each token as either "real" or "fake" based on whether the token was replaced by the generator. This part of the model is fine-tuned for downstream tasks after pretraining.

### Performance
- Typically outperforms BERT on NLP tasks. ELECTRA-small can perform similarly to BERT-based, and ELECTRA-base often surpasses BERT-base while being more efficient.


We'll be using ELECTRA-based, which is a standard-size model, comparable to BERT-base in terms of size and performance. It has 110M parameters.


In [25]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
import numpy as np
import random
import torch
from tqdm import tqdm

# Set random seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Initialize ELECTRA tokenizer
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

In [26]:
# Labels mapping
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values  # Convert sentiments to numeric labels

# Split dataset
train_d, val_d, train_labels, val_labels = train_test_split(data['processed_full_review'], y, test_size=0.2, random_state=42)
texts_train = list(train_d)
texts_val = list(val_d)

In [27]:
# Tokenize data
max_length = 64
tokenized_texts_train = tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
tokenized_texts_val = tokenizer(texts_val, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
train_labels = torch.tensor(list(train_labels))
val_labels = torch.tensor(list(val_labels))

In [28]:
# Create TensorDataset
train_dataset = TensorDataset(tokenized_texts_train['input_ids'], tokenized_texts_train['attention_mask'], train_labels)
val_dataset = TensorDataset(tokenized_texts_val['input_ids'], tokenized_texts_val['attention_mask'], val_labels)


In [29]:
# Initialize ELECTRA model for sequence classification with 3 output labels
model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=3)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# Set up optimizer, criterion, and learning rate scheduler
from sklearn.utils.class_weight import compute_class_weight
optimizer = AdamW(model.parameters(), lr=5e-6)
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
# Set batch size and DataLoader
from torch.amp import autocast, GradScaler

# Mixed precision scaler for faster training
scaler = GradScaler()
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2)


In [32]:
# Training parameters
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [34]:
# Initialize lists to accumulate overall metrics
train_losses = []
val_losses = []
val_accuracies = []
train_accuracies = []

train_precisions = []
train_recalls = []
train_f1s = []

val_precisions = []
val_recalls = []
val_f1s = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    tr_correct_preds = 0
    all_tr_labels = []
    all_tr_preds = []

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        # Use autocast for mixed precision
        with autocast(device_type="cuda"):
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            tr_loss = outputs.loss

        # Scale and backpropagate gradients
        scaler.scale(tr_loss).backward()
        
        # Clip gradients only when necessary
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        # Update training metrics
        train_loss += tr_loss.item()
        tr_logits = outputs.logits
        tr_preds = torch.argmax(tr_logits, dim=1)
        tr_correct_preds += torch.sum(tr_preds == labels).item()

        all_tr_labels.extend(labels.cpu().numpy())
        all_tr_preds.extend(tr_preds.cpu().numpy())

    scheduler.step()

    # Calculate average training loss and accuracy
    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_accuracy = tr_correct_preds / len(train_d)
    train_accuracies.append(train_accuracy)

    # Calculate precision, recall, F1 for training
    train_precision = precision_score(all_tr_labels, all_tr_preds, average='weighted')
    train_recall = recall_score(all_tr_labels, all_tr_preds, average='weighted')
    train_f1 = f1_score(all_tr_labels, all_tr_preds, average='weighted')

    # Accumulate train metrics
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1s.append(train_f1)

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            with autocast(device_type="cuda"):
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()

            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    # Calculate validation metrics
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = correct_preds / len(val_d)
    val_accuracies.append(val_accuracy)

    val_precision = precision_score(all_val_labels, all_val_preds, average='weighted')
    val_recall = recall_score(all_val_labels, all_val_preds, average='weighted')
    val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')

    # Accumulate validation metrics
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1s.append(val_f1)

    # Print metrics for current epoch
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Training Precision: {train_precision:.4f}, Training Recall: {train_recall:.4f}, Training F1: {train_f1:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")

# Calculate overall metrics after training
overall_train_loss = np.mean(train_losses)
overall_val_loss = np.mean(val_losses)
overall_train_accuracy = np.mean(train_accuracies)
overall_val_accuracy = np.mean(val_accuracies)

overall_train_precision = np.mean(train_precisions)
overall_train_recall = np.mean(train_recalls)
overall_train_f1 = np.mean(train_f1s)

overall_val_precision = np.mean(val_precisions)
overall_val_recall = np.mean(val_recalls)
overall_val_f1 = np.mean(val_f1s)

# Print overall performance metrics
print("\nOverall Performance Metrics:")
print(f"Average Training Loss: {overall_train_loss:.4f}, Average Training Accuracy: {overall_train_accuracy:.4f}")
print(f"Average Training Precision: {overall_train_precision:.4f}, Average Training Recall: {overall_train_recall:.4f}, Average Training F1: {overall_train_f1:.4f}")
print(f"Average Validation Loss: {overall_val_loss:.4f}, Average Validation Accuracy: {overall_val_accuracy:.4f}")
print(f"Average Validation Precision: {overall_val_precision:.4f}, Average Validation Recall: {overall_val_recall:.4f}, Average Validation F1: {overall_val_f1:.4f}")


Epoch 1/5 - Training: 100%|██████████| 72/72 [01:27<00:00,  1.21s/it]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch 1/5 - Validation: 100%|██████████| 18/18 [00:09<00:00,  1.87it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 1/5
Training Loss: 0.7992, Training Accuracy: 0.6858
Training Precision: 0.6270, Training Recall: 0.6858, Training F1: 0.5598
Validation Loss: 0.6663, Validation Accuracy: 0.7079
Validation Precision: 0.6810, Validation Recall: 0.7079, Validation F1: 0.5977


Epoch 2/5 - Training: 100%|██████████| 72/72 [01:31<00:00,  1.27s/it]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch 2/5 - Validation: 100%|██████████| 18/18 [00:09<00:00,  1.84it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 2/5
Training Loss: 0.6511, Training Accuracy: 0.7362
Training Precision: 0.6608, Training Recall: 0.7362, Training F1: 0.6662
Validation Loss: 0.6069, Validation Accuracy: 0.7721
Validation Precision: 0.6934, Validation Recall: 0.7721, Validation F1: 0.7153


Epoch 3/5 - Training: 100%|██████████| 72/72 [01:30<00:00,  1.25s/it]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch 3/5 - Validation: 100%|██████████| 18/18 [00:09<00:00,  1.88it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 4/5 - Training: 100%|██████████| 72/72 [01:30<00:00,  1.25s/it]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch 4/5 - Validation: 100%|██████████| 18/18 [00:09<00:00,  1.86it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 4/5
Training Loss: 0.6364, Training Accuracy: 0.7630
Training Precision: 0.6794, Training Recall: 0.7630, Training F1: 0.7080
Validation Loss: 0.6082, Validation Accuracy: 0.7708
Validation Precision: 0.6920, Validation Recall: 0.7708, Validation F1: 0.7136


Epoch 5/5 - Training: 100%|██████████| 72/72 [01:31<00:00,  1.27s/it]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch 5/5 - Validation: 100%|██████████| 18/18 [00:09<00:00,  1.81it/s]


Epoch 5/5
Training Loss: 0.6357, Training Accuracy: 0.7642
Training Precision: 0.6812, Training Recall: 0.7642, Training F1: 0.7097
Validation Loss: 0.6082, Validation Accuracy: 0.7708
Validation Precision: 0.6920, Validation Recall: 0.7708, Validation F1: 0.7136

Overall Performance Metrics:
Average Training Loss: 0.6725, Average Training Accuracy: 0.7425
Average Training Precision: 0.6657, Average Training Recall: 0.7425, Average Training F1: 0.6703
Average Validation Loss: 0.6196, Average Validation Accuracy: 0.7585
Average Validation Precision: 0.6901, Average Validation Recall: 0.7585, Average Validation F1: 0.6908



c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
